In [2]:
import pandas as pd
import datasets
from bs4 import BeautifulSoup
from io import StringIO
import csv
import torch
import difflib
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    AutoTokenizer,
    GenerationConfig,
    TextStreamer,
    pipeline,
)


from tqdm import tqdm
import pandas as pd
import re

data = pd.read_csv('../data/pseudo_json_dataset_V3_2048.csv')

/home/ubuntu/llm/Large-Language-Model-for-Key-Value-Extractions/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/llm/Large-Language-Model-for-Key-Value-Extractions/.venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ubuntu/llm/Large-Language-Model-for-Key-Value-Extractions/.venv/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ubuntu/llm/Large-Language-Model-for-Key-Value-Extractions/.venv/lib/python3.10/sit

In [3]:
model_id = 'RUCKBReasoning/TableLLM-13b'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_id, device_map="auto", torch_dtype=torch.float16, load_in_8bit=True
)
generation_config = GenerationConfig.from_pretrained(model_id)
print(generation_config)


generation_config.max_new_tokens = 512
generation_config.temperature = 0.0001
generation_config.do_sample = True
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

Loading checkpoint shards:  50%|█████     | 3/6 [01:50<01:50, 36.85s/it]

In [ ]:
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    # streamer=streamer,
)


In [ ]:
def extract(ip):
    # Find the index of "[/INST]"
    index = ip.find("[/INST]")

    # Extract the value after "[/INST]"
    if index != -1:
        extracted_value = ip[index + len("[/INST]") + 1:].strip()  # Add 1 to skip the "]" character
        return extracted_value
    else:
        print("No value found after '[/INST]'")

def extract_second(ip):
   # Find the first occurrence of "[/INST]"
    first_inst_index = ip.find("[/INST]")

    # Find the second occurrence of "[/INST]" starting from the position after the first one
    second_inst_index = ip.find("[/INST]", first_inst_index + len("[/INST]"))

    # Extract the value after the second "[/INST]"
    if second_inst_index != -1:
        extracted_value = ip[second_inst_index + len("[/INST]") + 1:].strip()  # Add 1 to skip the "]" character
        return extracted_value
    else:
        print("Two occurrences of '[/INST]' not found")


In [ ]:
output_df = pd.DataFrame(columns=['csv', 'question', 'answer', 'prediction'])

In [ ]:
def prompt_formatter(csv_data, question):
    ip = f"""[INST]Offer a thorough and accurate solution that directly addresses the Question outlined in the [Question].

    ### [Table]
    {csv_data}

    ### [Question]
    {question}

    ### [Solution]
    [/INST]"""
    return ip 

In [ ]:
def twoshot_prompt_formatter(csv_data, question):
    ip = f"""[INST]Offer a thorough and accurate solution that directly addresses the Question outlined in the [Question].

    ### [Table]
    1,-,2002,2001,2000
    2,net sales,$ 5742,$ 5363,$ 7983
    3,cost of sales,4139,4128,5817
    4,gross margin,$ 1603,$ 1235,$ 2166
    5,gross margin percentage,28% ( 28 % ),23% ( 23 % ),27% ( 27 % )


    ### [Question]
    what is the gross margin in 2002?

    ### [Solution]
    [/INST] The gross margin in 2002 is $1603.

    [INST]Offer a thorough and accurate solution that directly addresses the Question outlined in the [Question].

    ### [Table]
    {csv_data}

    ### [Question]
    {question}

    ### [Solution]
    [/INST]"""
    return ip 

In [ ]:
filename = 'tableLLM_output.txt'

In [ ]:
# with open(filename, 'w') as file:
for i, row in tqdm(data[10:50].iterrows()):
    soup = BeautifulSoup(row['html_content'], 'html.parser')

    # Find the table
    table = soup.find('table')
    # Open StringIO object for writing
    csv_buffer = StringIO()


    # Write rows to the StringIO object
    writer = csv.writer(csv_buffer)
    trows = table.find_all('tr')
    for trow in trows:
        writer.writerow([cell.get_text() for cell in trow.find_all(['td', 'th'])])


    # Get the CSV data as a string
    csv_data = csv_buffer.getvalue()

    ip = twoshot_prompt_formatter(csv_data, row['question'])

    output = llm(ip)[0]['generated_text']
    print(i)
    print(row['answer'])

    pred = extract_second(output) 
    print(pred)
    print('***')
    output_df.at[i, 'csv'] = csv_data
    output_df.at[i, 'question'] = data.loc[i,'question']
    output_df.at[i, 'answer'] = data.loc[i,'answer']
    output_df.at[i, 'prediction'] = pred

NameError: name 'tqdm' is not defined

In [ ]:
output_df.to_csv('test_one_shot.csv', index=False)

In [ ]:

    
import os
 
# Get the process ID of the current Jupyter Notebook server
pid = os.getpid()
pid

11719

: 